In [156]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
# dependencies
import pymongo as pm

In [181]:
def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [160]:
def feature_extractor(html):
    '''
    Parse html using newspaper
    
    Parameters:
    --------
    html   : 'string'
    
    Returns:
    --------
    result : dictionary
    
    '''
    import newspaper
    from bs4 import BeautifulSoup as bs
    from datetime import datetime
    
    article = newspaper.Article('')
    article.set_html(html)
    article.build()
    
    # parse date manually if it wasn't found by newspaper
    if not article.publish_date:
        sp = bs(html, 'lxml')
        # 'dek___3AQpw' class appears on 30% of msnbc websites
        try:
            publish_date = datetime.strptime(\
                               (sp.find('p', class_='dek___3AQpw').span.text),\
                               '%b.%d.%Y'\
                            )
        except:
            publish_date = ''
    else:
        publish_date = article.publish_date


    return {
            'date'    :publish_date,
            'title'   :article.title,
            'text'    :article.text,
            'authors' :article.authors,
            'keywords':article.keywords
    }

In [175]:
def docs_parser(htmlCol, skip=True):
    '''
    Parse mongo docs, extract features and update the doc with the features
    
    Parameters:
    --------
    htmlCol : mongodb collection, has to have documents with 'html' key
    skip    : skip html processing if meta key exists in a record, default "True"
    
    Returns:
    --------
    updates all documents in the collection
    '''

    for doc in htmlCol.find():
        print(f"{doc['_id']}:")
            
        if 'html' in doc:
            if 'meta' in doc and skip: 
                
            # extract metadata from html
            meta = feature_extractor(doc['html'])
            
            try:
                if meta == doc['meta']:
                    print(f"has same meta")
            except:
                # if there is newer meta data or meta key is not existing
                htmlCol.update_one(
                    {'_id':ObjectId(doc['_id'])},
                    {'$set' : {
                              'meta' : meta
                              }
                    }
                )
                print(f"updated meta")
        else:
            print(f"does not have html")
        print('----------')

In [162]:
def show_doc(id):
    '''
    Finds a document by 'id' and prints contents to the console
    
    Parameters:
    --------
    id : mongodb document id
    
    Returns:
    --------
    Prints first 100 symbols of each document's key to console
    '''
    from bson.objectid import ObjectId
    doc = htmlCol.find_one({'_id':ObjectId('5b243fbc897e82028b8ffe52')})
    for k in doc:
        print(f"{k} : {str(doc[k])[:100]}")

Some comands to keep dbs clean

In [98]:
# deletes all 'meta' fields from all docs
# htmlCol.update({}, {$unset: {meta:1}}, false, true); # mongo shell comand
htmlCol.update({}, {'$unset': {'meta':1}}, multi=True) # pymongo way


/Users/yegor3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 1586, 'nModified': 1586, 'ok': 1.0, 'updatedExisting': True}

In [105]:
# leaves only unique documents by 'url' field

htmlCol.create_index(
    "url",
    unique=True
)

'url_1'

In [ ]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [127]:
# find documents NOT containing a 'tag': regex expression
import re
tag = re.compile('dek___3AQpw.')
docs = htmlCol.find({"html" : {'$not': tag}})
for d in docs[:20]: print(d['url'])


http://www.msnbc.com/
http://www.msnbc.com/{{path.prefix}}/transcripts
http://www.msnbc.com/home2?page=1
http://www.msnbc.com/rachel-maddow-show/trump-jr-offers-the-wrong-response-concerns-the-gop-cult
http://www.msnbc.com/explore
http://www.msnbc.com/rachel-maddow-show/president-tries-brush-lying-about-infamous-trump-tower-meeting
http://www.msnbc.com/#main-menu
http://www.msnbc.com/{{path.prefix}}/transcripts#main-menu
https://www.facebook.com/sharer/sharer.php?u=https://www.msnbc.com/the-last-word/watch/lawrence-trump-tries-to-steal-the-grief-of-fallen-soldiers-parents-1256235075972
http://www.msnbc.com/rachel-maddow-show/the-good-question-team-trump-considers-ridiculous-and-ludicrous
http://www.msnbc.com/guns
https://twitter.com/intent/tweet?text=Trump%20calls%20IG%20report%20'horror%20show'%2C%20blames%20Dems%20for%20separations&via=msnbc&url=https://www.msnbc.com/morning-joe/watch/trump-calls-ig-report-horror-show-blames-dems-for-separations-1256477251672&original_referer=https:/

In [155]:
??test

In [ ]:
test = newspaper.Article(url='http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump')
test.build()

In [146]:
print(test.publish_date)

None


In [150]:
test.date

AttributeError: 'Article' object has no attribute 'date'

In [147]:
test.meta_data

defaultdict(dict,
            {'og': {'url': 'http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump',
              'type': 'article',
              'site_name': 'MSNBC',
              'title': 'Defamation lawsuit grows more serious for Trump',
              'description': "As if Donald Trump's lawyers weren't already busy, Summer Zervos' civil suit creates a real threat -- which isn't going away.",
              'image': 'http://www.msnbc.com/sites/msnbc/files/styles/ratio--1_91-1--1200x630/public/06525535.jpg?itok=lrFK_ych'},
             'twitter': {'card': 'summary_large_image',
              'site': '@msnbc',
              'creator': {'identifier': '@msnbc', 'id': 2836421},
              'url': 'http://www.msnbc.com/rachel-maddow-show/defamation-lawsuit-grows-more-serious-trump',
              'account_id': 2836421,
              'domain': 'msnbc.com',
              'title': 'Defamation lawsuit grows more serious for Trump',
              'description

# **Production code**

In [193]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

# define db 
DB_NAME = 'scrape'
db = client[DB_NAME]

In [199]:
# import json files into db
path = 'db/json/by_media/'
left = 'left/'
lefts = !ls {path + left}

for file in lefts: 
    print(file)
    addToDB(DB_NAME, left[:-1], path+left, file)

DailyBeast2forLinks.json
2018-06-26T22:26:03.353-0400	connected to: localhost
2018-06-26T22:26:04.768-0400	imported 829 documents
Collection left in scrape database created
MotherJones2forLinks.json
2018-06-26T22:26:04.932-0400	connected to: localhost
2018-06-26T22:26:07.922-0400	[#######.................] scrape.left	168MB/525MB (31.9%)
2018-06-26T22:26:10.920-0400	[#############...........] scrape.left	296MB/525MB (56.3%)
2018-06-26T22:26:13.923-0400	[####################....] scrape.left	455MB/525MB (86.6%)
2018-06-26T22:26:16.400-0400	[########################] scrape.left	525MB/525MB (100.0%)
2018-06-26T22:26:16.400-0400	imported 4681 documents
Collection left in scrape database created
NewRepublic2forLinks.json
2018-06-26T22:26:16.595-0400	connected to: localhost
2018-06-26T22:26:17.422-0400	imported 444 documents
Collection left in scrape database created
Slate2forLinks.json
2018-06-26T22:26:17.592-0400	connected to: localhost
2018-06-26T22:26:20.574-0400	[######................

In [202]:
# import json files into db
path = 'db/json/by_media/'
right = 'right/'
rights = !ls {path + right}

for file in rights: 
    print(file)
    addToDB(DB_NAME, right[:-1], path+right, file)

Hill_scrape_db.json
2018-06-26T22:29:01.374-0400	connected to: localhost
2018-06-26T22:29:04.363-0400	[#######.................] scrape.right	384MB/1.15GB (32.7%)
2018-06-26T22:29:07.363-0400	[###############.........] scrape.right	767MB/1.15GB (65.3%)
2018-06-26T22:29:10.362-0400	[######################..] scrape.right	1.05GB/1.15GB (91.9%)
2018-06-26T22:29:11.422-0400	[########################] scrape.right	1.15GB/1.15GB (100.0%)
2018-06-26T22:29:11.422-0400	imported 11985 documents
Collection right in scrape database created
REAS_scrape_db.json
2018-06-26T22:29:11.601-0400	connected to: localhost
2018-06-26T22:29:14.592-0400	[######..................] scrape.right	379MB/1.35GB (27.5%)
2018-06-26T22:29:17.589-0400	[############............] scrape.right	744MB/1.35GB (53.9%)
2018-06-26T22:29:20.590-0400	[##################......] scrape.right	1.06GB/1.35GB (78.8%)
2018-06-26T22:29:23.589-0400	[#######################.] scrape.right	1.29GB/1.35GB (96.0%)
2018-06-26T22:29:24.267-0400	[#

In [208]:
for collection in ['left','right']: docs_parser(db[collection])

5b2921ade9541e01584077b3:
updated meta
----------
5b2921a7e9541e01584077b2:
updated meta
----------
5b2921b3e9541e01584077b4:
updated meta
----------
5b2921a1e9541e01584077b1:
updated meta
----------
5b2921b9e9541e01584077b5:
updated meta
----------
5b2921c5e9541e01584077b7:
updated meta
----------
5b2921bfe9541e01584077b6:
updated meta
----------
5b2921cbe9541e01584077b8:
updated meta
----------
5b2921d6e9541e01584077ba:
updated meta
----------
5b2921d0e9541e01584077b9:
updated meta
----------
5b2921dce9541e01584077bb:
updated meta
----------
5b2921e2e9541e01584077bc:
updated meta
----------
5b2921eee9541e01584077be:
updated meta
----------
5b2921e8e9541e01584077bd:
updated meta
----------
5b2921f4e9541e01584077bf:
updated meta
----------
5b2921fae9541e01584077c0:
updated meta
----------
5b292206e9541e01584077c2:
updated meta
----------
5b2921ffe9541e01584077c1:
updated meta
----------
5b29220ce9541e01584077c3:
updated meta
----------
5b292218e9541e01584077c5:
updated meta
----------


updated meta
----------
5b29259fe9541e015840785a:
updated meta
----------
5b2925a5e9541e015840785b:
updated meta
----------
5b2925abe9541e015840785c:
updated meta
----------
5b2925b1e9541e015840785d:
updated meta
----------
5b2925b6e9541e015840785e:
updated meta
----------
5b2925bde9541e015840785f:
updated meta
----------
5b2925c9e9541e0158407861:
updated meta
----------
5b2925d5e9541e0158407863:
updated meta
----------
5b2925dbe9541e0158407864:
updated meta
----------
5b2925e2e9541e0158407865:
updated meta
----------
5b2925e7e9541e0158407866:
updated meta
----------
5b2925c3e9541e0158407860:
updated meta
----------
5b2925f4e9541e0158407868:
updated meta
----------
5b2925cfe9541e0158407862:
updated meta
----------
5b2925eee9541e0158407867:
updated meta
----------
5b2925fae9541e0158407869:
updated meta
----------
5b292607e9541e015840786a:
updated meta
----------
5b292625e9541e015840786f:
updated meta
----------
5b292619e9541e015840786d:
updated meta
----------
5b29262be9541e0158407870:


updated meta
----------
5b292982e9541e01584078fe:
updated meta
----------
5b29298ee9541e0158407900:
updated meta
----------
5b292994e9541e0158407901:
updated meta
----------
5b29299ae9541e0158407902:
updated meta
----------
5b292988e9541e01584078ff:
updated meta
----------
5b2929a0e9541e0158407903:
updated meta
----------
5b2929a7e9541e0158407904:
updated meta
----------
5b2929ade9541e0158407905:
updated meta
----------
5b2929c6e9541e0158407909:
updated meta
----------
5b2929cce9541e015840790a:
updated meta
----------
5b2929bbe9541e0158407907:
updated meta
----------
5b2929c1e9541e0158407908:
updated meta
----------
5b2929d2e9541e015840790b:
updated meta
----------
5b2929d7e9541e015840790c:
updated meta
----------
5b2929b5e9541e0158407906:
updated meta
----------
5b2929e8e9541e015840790e:
updated meta
----------
5b2929e2e9541e015840790d:
updated meta
----------
5b2929ede9541e015840790f:
updated meta
----------
5b2929f3e9541e0158407910:
updated meta
----------
5b2929fce9541e0158407911:


updated meta
----------
5b292d83e9541e01584079a1:
updated meta
----------
5b292d9be9541e01584079a5:
updated meta
----------
5b292daee9541e01584079a8:
updated meta
----------
5b292db3e9541e01584079a9:
updated meta
----------
5b292db9e9541e01584079aa:
updated meta
----------
5b292d95e9541e01584079a4:
updated meta
----------
5b292da8e9541e01584079a7:
updated meta
----------
5b292dbee9541e01584079ab:
updated meta
----------
5b292dcae9541e01584079ad:
updated meta
----------
5b292dc4e9541e01584079ac:
updated meta
----------
5b292de1e9541e01584079b1:
updated meta
----------
5b292de7e9541e01584079b2:
updated meta
----------
5b292da1e9541e01584079a6:
updated meta
----------
5b292dd0e9541e01584079ae:
updated meta
----------
5b292ddce9541e01584079b0:
updated meta
----------
5b292dd6e9541e01584079af:
updated meta
----------
5b292dede9541e01584079b3:
updated meta
----------
5b292df2e9541e01584079b4:
updated meta
----------
5b292e01e9541e01584079b6:
updated meta
----------
5b292dfbe9541e01584079b5:


updated meta
----------
5b293191e9541e0158407a4b:
updated meta
----------
5b29318ce9541e0158407a4a:
updated meta
----------
5b293197e9541e0158407a4c:
updated meta
----------
5b29317ae9541e0158407a47:
updated meta
----------
5b2931b0e9541e0158407a50:
updated meta
----------
5b2931bce9541e0158407a52:
updated meta
----------
5b29319ee9541e0158407a4d:
updated meta
----------
5b2931a4e9541e0158407a4e:
updated meta
----------
5b2931b6e9541e0158407a51:
updated meta
----------
5b2931c2e9541e0158407a53:
updated meta
----------
5b2931abe9541e0158407a4f:
updated meta
----------
5b2931e6e9541e0158407a59:
updated meta
----------
5b2931e0e9541e0158407a58:
updated meta
----------
5b2931cfe9541e0158407a55:
updated meta
----------
5b2931dbe9541e0158407a57:
updated meta
----------
5b2931ece9541e0158407a5a:
updated meta
----------
5b2931d5e9541e0158407a56:
updated meta
----------
5b2931c8e9541e0158407a54:
updated meta
----------
5b293200e9541e0158407a5d:
updated meta
----------
5b2931fae9541e0158407a5c:


...

updated meta
----------
5b29828ee9541e20f0757b4f:
updated meta
----------
5b298295e9541e20f0757b50:
updated meta
----------
5b29829ce9541e20f0757b51:
updated meta
----------
5b2982a2e9541e20f0757b52:
updated meta
----------
5b2982a8e9541e20f0757b53:
updated meta
----------
5b2982aee9541e20f0757b54:
updated meta
----------
5b2982b4e9541e20f0757b55:
updated meta
----------
5b2982c0e9541e20f0757b57:
updated meta
----------
5b2982bae9541e20f0757b56:
updated meta
----------
5b2982cde9541e20f0757b59:
updated meta
----------
5b2982c6e9541e20f0757b58:
updated meta
----------
5b2982d3e9541e20f0757b5a:
updated meta
----------
5b2982e5e9541e20f0757b5d:
updated meta
----------
5b2982d9e9541e20f0757b5b:
updated meta
----------
5b2982f3e9541e20f0757b5f:
updated meta
----------
5b2982ffe9541e20f0757b61:
updated meta
----------
5b2982f9e9541e20f0757b60:
updated meta
----------
5b298306e9541e20f0757b62:
updated meta
----------
5b2982e0e9541e20f0757b5c:
updated meta
----------
5b2982ece9541e20f0757b5e:


updated meta
----------
5b2986b5e9541e20f0757bf4:
updated meta
----------
5b2986afe9541e20f0757bf3:
updated meta
----------
5b2986c2e9541e20f0757bf6:
updated meta
----------
5b2986c8e9541e20f0757bf7:
updated meta
----------
5b2986bbe9541e20f0757bf5:
updated meta
----------
5b2986d3e9541e20f0757bf8:
updated meta
----------
5b2986e0e9541e20f0757bfa:
updated meta
----------
5b2986d9e9541e20f0757bf9:
updated meta
----------
5b2986ece9541e20f0757bfc:
updated meta
----------
5b2986e6e9541e20f0757bfb:
updated meta
----------
5b2986f3e9541e20f0757bfd:
updated meta
----------
5b2986fee9541e20f0757bfe:
updated meta
----------
5b298709e9541e20f0757bff:
updated meta
----------
5b298717e9541e20f0757c01:
updated meta
----------
5b29871de9541e20f0757c02:
updated meta
----------
5b298710e9541e20f0757c00:
updated meta
----------
5b298722e9541e20f0757c03:
updated meta
----------
5b29872fe9541e20f0757c05:
updated meta
----------
5b298729e9541e20f0757c04:
updated meta
----------
5b298736e9541e20f0757c06:


updated meta
----------
5b298aede9541e20f0757c97:
updated meta
----------
5b298b00e9541e20f0757c9a:
updated meta
----------
5b298af9e9541e20f0757c99:
updated meta
----------
5b298b06e9541e20f0757c9b:
updated meta
----------
5b298b0ce9541e20f0757c9c:
updated meta
----------
5b298b19e9541e20f0757c9e:
updated meta
----------
5b298b13e9541e20f0757c9d:
updated meta
----------
5b298b20e9541e20f0757c9f:
updated meta
----------
5b298b32e9541e20f0757ca1:
updated meta
----------
5b298b3ee9541e20f0757ca3:
updated meta
----------
5b298b38e9541e20f0757ca2:
updated meta
----------
5b298b2be9541e20f0757ca0:
updated meta
----------
5b298b68e9541e20f0757ca9:
updated meta
----------
5b298b4ce9541e20f0757ca5:
updated meta
----------
5b298b53e9541e20f0757ca6:
updated meta
----------
5b298b45e9541e20f0757ca4:
updated meta
----------
5b298b5ae9541e20f0757ca7:
updated meta
----------
5b298b61e9541e20f0757ca8:
updated meta
----------
5b298b80e9541e20f0757cad:
updated meta
----------
5b298b6ee9541e20f0757caa:


updated meta
----------
5b298f4fe9541e20f0757d3c:
updated meta
----------
5b298f55e9541e20f0757d3d:
updated meta
----------
5b298f5ce9541e20f0757d3e:
updated meta
----------
5b298f71e9541e20f0757d3f:
updated meta
----------
5b298f77e9541e20f0757d40:
updated meta
----------
5b298f8ce9541e20f0757d43:
updated meta
----------
5b298f7ee9541e20f0757d41:
updated meta
----------
5b298f85e9541e20f0757d42:
updated meta
----------
5b298f97e9541e20f0757d45:
You must `parse()` an article first!


ArticleException: 

In [122]:
show_doc('5b243fb2897e82028b8ffe50')

_id : 5b243fbc897e82028b8ffe52
url : https://www.msnbc.com/all-in/watch/white-house-can-t-fill-open-positions-turns-to-job-fair-125617056
html : <!DOCTYPE html><html lang="en" data-reactroot=""><head><title data-rh="true">White House can&#x27;t 
